## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split , RandomizedSearchCV , GridSearchCV
from sklearn.svm import SVC , LinearSVC
from sklearn.metrics import classification_report
import m2cgen
import os
import matplotlib.pyplot as plt
import joblib
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 1000)  # Set max width
import sys
import threading

Activity Mapper:

0 : walking
1 : jogging
2 : upstairs
3 : downstairs
4 : sitting
5 : standing
6 : sleeping

## Load Data

In [2]:
dataset_path = os.getcwd().replace('/notebook' , '') + "/dataset/"

# Read training data
imu_df = pd.read_csv(dataset_path + "imu_data_v2.csv")
imu_df = imu_df.iloc[:,1:]
imu_df.columns = imu_df.columns.str.replace("'" , "")
x = imu_df.iloc[:,:-1]
y = imu_df.iloc[:,-1]
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

In [3]:
x.columns

Index(['attitude.roll', ' attitude.pitch', ' attitude.yaw', ' gravity.x', ' gravity.y', ' gravity.z', ' rotationRate.x', ' rotationRate.y', ' rotationRate.z', ' userAcceleration.x', ' userAcceleration.y', ' userAcceleration.z', ' magnetic.x', ' magnetic.y', ' magnetic.z', ' temperature'], dtype='object')

# Modeling

## Motion Classification

In [4]:
"""
Map
0 - steady motion,
1 - unsteady motion
"""
mapper = {
    0 : 1,
    1 : 1,
    2 : 1,
    3 : 1,
    4 : 0,
    5 : 0,
    6 : 0
}

In [5]:
motion_y = y.map(mapper)

x1, x_test , y1 , y_test = train_test_split(x_scaled ,motion_y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [6]:
svc = LinearSVC()

param_dist = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge', 'hinge']
    }

random_search = GridSearchCV(estimator = svc,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/

,mean_test_score,params
11,0.973750,"{'C': 10, 'loss': 'hinge', 'penalty': 'l2'}"
15,0.972049,"{'C': 100, 'loss': 'hinge', 'penalty': 'l2'}"
9,0.971806,"{'C': 10, 'loss': 'squared_hinge', 'penalty': 'l2'}"
7,0.971719,"{'C': 1, 'loss': 'hinge', 'penalty': 'l2'}"
5,0.971042,"{'C': 1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
13,0.968455,"{'C': 100, 'loss': 'squared_hinge', 'penalty': 'l2'}"
1,0.967674,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
3,0.963628,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l2'}"
0,NaN,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l1'}"
2,NaN,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l1'}"


In [7]:
svc = LinearSVC(C = 10, loss= 'hinge' , penalty= 'l2')
svc.fit(x_train , y_train)
svc.score(x_val , y_val)
svc_report = classification_report(svc.predict(x_val) , y_val)
print(svc_report)

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      6826
           1       0.97      0.98      0.98      7574

    accuracy                           0.97     14400
   macro avg       0.97      0.97      0.97     14400
weighted avg       0.97      0.97      0.97     14400



/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [8]:
svc_train_report = classification_report(svc.predict(x_train) , y_train)
print(svc_train_report)

              precision    recall  f1-score   support

           0       0.98      0.96      0.97     27387
           1       0.97      0.98      0.98     30213

    accuracy                           0.97     57600
   macro avg       0.97      0.97      0.97     57600
weighted avg       0.97      0.97      0.97     57600



In [9]:
svc_test_report = classification_report(svc.predict(x_test) , y_test)
print(svc_test_report)

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      8553
           1       0.97      0.98      0.98      9447

    accuracy                           0.97     18000
   macro avg       0.97      0.97      0.97     18000
weighted avg       0.97      0.97      0.97     18000



In [10]:
with open(os.getcwd().replace('/notebook' , '') + "/model/motion_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(svc))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/motion_model.py" ))

569


## Steady Movement Classification

In [11]:
steady_df  =imu_df[(imu_df['Target'] == 4) | (imu_df['Target'] == 5) | (imu_df['Target'] == 6)]
x = steady_df.iloc[:,:-1]
y = steady_df.iloc[:,-1]

scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

x1, x_test , y1 , y_test = train_test_split(x_scaled , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [12]:
svc = LinearSVC()

param_dist = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge', 'hinge']
    }

random_search = GridSearchCV(estimator = svc,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/

,mean_test_score,params
9,0.773028,"{'C': 10, 'loss': 'squared_hinge', 'penalty': 'l2'}"
5,0.771540,"{'C': 1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
1,0.768006,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
15,0.762016,"{'C': 100, 'loss': 'hinge', 'penalty': 'l2'}"
11,0.754799,"{'C': 10, 'loss': 'hinge', 'penalty': 'l2'}"
3,0.753795,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l2'}"
7,0.748065,"{'C': 1, 'loss': 'hinge', 'penalty': 'l2'}"
13,0.719345,"{'C': 100, 'loss': 'squared_hinge', 'penalty': 'l2'}"
0,NaN,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l1'}"
2,NaN,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l1'}"


In [13]:
svc = LinearSVC(C = 10, loss= 'squared_hinge' , penalty= 'l2')
svc.fit(x_train , y_train)
svc.score(x_val , y_val)
svc_report = classification_report(svc.predict(x_val) , y_val)
print(svc_report)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


              precision    recall  f1-score   support

           4       0.69      0.66      0.67      2366
           5       0.64      0.68      0.66      2087
           6       1.00      0.98      0.99      2267

    accuracy                           0.78      6720
   macro avg       0.78      0.77      0.77      6720
weighted avg       0.78      0.78      0.78      6720



/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [14]:
svc_train_report = classification_report(svc.predict(x_train) , y_train)
print(svc_train_report)

              precision    recall  f1-score   support

           4       0.69      0.66      0.67      9357
           5       0.64      0.68      0.66      8426
           6       0.99      0.98      0.99      9097

    accuracy                           0.77     26880
   macro avg       0.77      0.77      0.77     26880
weighted avg       0.78      0.77      0.77     26880



In [15]:
svc_test_report = classification_report(svc.predict(x_test) , y_test)
print(svc_test_report)

              precision    recall  f1-score   support

           4       0.67      0.66      0.66      2872
           5       0.64      0.67      0.65      2687
           6       1.00      0.98      0.99      2841

    accuracy                           0.77      8400
   macro avg       0.77      0.77      0.77      8400
weighted avg       0.77      0.77      0.77      8400



In [16]:
with open(os.getcwd().replace('/notebook' , '') + "/model/steady_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(svc))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/steady_model.py" ))

1661


## Unsteady Movement Classification

In [17]:
unsteady_df  =imu_df[(imu_df['Target'] != 4) & (imu_df['Target'] != 5) & (imu_df['Target'] != 6)]
x = unsteady_df.iloc[:,:-1]
y = unsteady_df.iloc[:,-1]

scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

## Unsteady Motion Classification

In [18]:
"""
Map
0 - surface motion,
1 - staircase motion
"""
mapper = {
    0 : 0,
    1 : 0,
    2 : 1,
    3 : 1
}

In [19]:
unsteady_motion_y = y.map(mapper)

x1, x_test , y1 , y_test = train_test_split(x_scaled ,unsteady_motion_y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [20]:
svc = LinearSVC()

param_dist = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge', 'hinge']
    }

random_search = GridSearchCV(estimator = svc,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/

,mean_test_score,params
11,0.836947,"{'C': 10, 'loss': 'hinge', 'penalty': 'l2'}"
15,0.835091,"{'C': 100, 'loss': 'hinge', 'penalty': 'l2'}"
7,0.833919,"{'C': 1, 'loss': 'hinge', 'penalty': 'l2'}"
9,0.831380,"{'C': 10, 'loss': 'squared_hinge', 'penalty': 'l2'}"
5,0.830794,"{'C': 1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
1,0.828711,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
3,0.826497,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l2'}"
13,0.801204,"{'C': 100, 'loss': 'squared_hinge', 'penalty': 'l2'}"
0,NaN,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l1'}"
2,NaN,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l1'}"


In [21]:
svc = LinearSVC(C = 10, loss= 'hinge' , penalty= 'l2')
svc.fit(x_train , y_train)
svc.score(x_val , y_val)
svc_report = classification_report(svc.predict(x_val) , y_val)
print(svc_report)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.81      0.85      0.83      3689
           1       0.85      0.82      0.83      3991

    accuracy                           0.83      7680
   macro avg       0.83      0.83      0.83      7680
weighted avg       0.83      0.83      0.83      7680



In [22]:
svc_train_report = classification_report(svc.predict(x_train) , y_train)
print(svc_train_report)

              precision    recall  f1-score   support

           0       0.83      0.84      0.84     15112
           1       0.85      0.83      0.84     15608

    accuracy                           0.84     30720
   macro avg       0.84      0.84      0.84     30720
weighted avg       0.84      0.84      0.84     30720



In [23]:
svc_test_report = classification_report(svc.predict(x_test) , y_test)
print(svc_test_report)

              precision    recall  f1-score   support

           0       0.83      0.84      0.84      4765
           1       0.84      0.84      0.84      4835

    accuracy                           0.84      9600
   macro avg       0.84      0.84      0.84      9600
weighted avg       0.84      0.84      0.84      9600



In [24]:
with open(os.getcwd().replace('/notebook' , '') + "/model/unsteady_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(svc))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/unsteady_model.py" ))

569


## Staircase motion

In [25]:
staircase_df  =imu_df[(imu_df['Target'] == 1) | (imu_df['Target'] == 2)]
x = staircase_df.iloc[:,:-1]
y = staircase_df.iloc[:,-1]

scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

x1, x_test , y1 , y_test = train_test_split(x_scaled , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [26]:
svc = LinearSVC()

param_dist = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge', 'hinge']
    }


random_search = GridSearchCV(estimator = svc,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/

,mean_test_score,params
15,0.835938,"{'C': 100, 'loss': 'hinge', 'penalty': 'l2'}"
11,0.824284,"{'C': 10, 'loss': 'hinge', 'penalty': 'l2'}"
7,0.811328,"{'C': 1, 'loss': 'hinge', 'penalty': 'l2'}"
9,0.805013,"{'C': 10, 'loss': 'squared_hinge', 'penalty': 'l2'}"
5,0.803320,"{'C': 1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
1,0.800130,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
3,0.798177,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l2'}"
13,0.793685,"{'C': 100, 'loss': 'squared_hinge', 'penalty': 'l2'}"
0,NaN,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l1'}"
2,NaN,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l1'}"


In [27]:
svc = LinearSVC(C = 100, loss= 'hinge' , penalty= 'l2')
svc.fit(x_train , y_train)
svc.score(x_val , y_val)
svc_report = classification_report(svc.predict(x_val) , y_val)
print(svc_report)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


              precision    recall  f1-score   support

           1       0.74      0.91      0.82      1561
           2       0.93      0.78      0.85      2279

    accuracy                           0.83      3840
   macro avg       0.83      0.85      0.83      3840
weighted avg       0.85      0.83      0.84      3840



/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [28]:
svc_train_report = classification_report(svc.predict(x_train) , y_train)
print(svc_train_report)

              precision    recall  f1-score   support

           1       0.74      0.92      0.82      6164
           2       0.94      0.78      0.85      9196

    accuracy                           0.84     15360
   macro avg       0.84      0.85      0.84     15360
weighted avg       0.86      0.84      0.84     15360



In [29]:
svc_test_report = classification_report(svc.predict(x_test) , y_test)
print(svc_test_report)

              precision    recall  f1-score   support

           1       0.74      0.91      0.82      1949
           2       0.93      0.78      0.85      2851

    accuracy                           0.84      4800
   macro avg       0.84      0.85      0.83      4800
weighted avg       0.85      0.84      0.84      4800



In [30]:
with open(os.getcwd().replace('/notebook' , '') + "/model/staircase_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(svc))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/staircase_model.py" ))

566


## Surface motion classification

In [31]:
surface_df  =imu_df[(imu_df['Target'] == 0) | (imu_df['Target'] == 4)]
x = surface_df.iloc[:,:-1]
y = surface_df.iloc[:,-1]

scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

x1, x_test , y1 , y_test = train_test_split(x_scaled , y ,test_size= 0.2, stratify = y)

x_train , x_val , y_train , y_val = train_test_split(x1 ,y1 ,test_size= 0.2, stratify = y1)

### Modelling

In [32]:
svc = LinearSVC()

param_dist = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge', 'hinge']
    }

random_search = GridSearchCV(estimator = svc,
                            param_grid = param_dist,
                            cv = 5,
                            n_jobs=-1
                            )
random_search.fit(x_train, y_train)

cv_results = pd.DataFrame(random_search.cv_results_)
cv_results = cv_results[["mean_test_score", "params"]].sort_values(by="mean_test_score", ascending=False)

cv_results 

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/

,mean_test_score,params
13,1.000000,"{'C': 100, 'loss': 'squared_hinge', 'penalty': 'l2'}"
15,1.000000,"{'C': 100, 'loss': 'hinge', 'penalty': 'l2'}"
5,0.999940,"{'C': 1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
7,0.999940,"{'C': 1, 'loss': 'hinge', 'penalty': 'l2'}"
9,0.999940,"{'C': 10, 'loss': 'squared_hinge', 'penalty': 'l2'}"
11,0.999940,"{'C': 10, 'loss': 'hinge', 'penalty': 'l2'}"
1,0.999519,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l2'}"
3,0.999399,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l2'}"
0,NaN,"{'C': 0.1, 'loss': 'squared_hinge', 'penalty': 'l1'}"
2,NaN,"{'C': 0.1, 'loss': 'hinge', 'penalty': 'l1'}"


In [33]:
svc = svc = LinearSVC(C = 100, loss= 'squared_hinge' , penalty= 'l2')
svc.fit(x_train , y_train)
svc.score(x_val , y_val)
svc_report = classification_report(svc.predict(x_val) , y_val)
print(svc_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1920
           4       1.00      1.00      1.00      2240

    accuracy                           1.00      4160
   macro avg       1.00      1.00      1.00      4160
weighted avg       1.00      1.00      1.00      4160



/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [34]:
svc_train_report = classification_report(svc.predict(x_train) , y_train)
print(svc_train_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7680
           4       1.00      1.00      1.00      8960

    accuracy                           1.00     16640
   macro avg       1.00      1.00      1.00     16640
weighted avg       1.00      1.00      1.00     16640



In [35]:
svc_test_report = classification_report(svc.predict(x_test) , y_test)
print(svc_test_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2400
           4       1.00      1.00      1.00      2800

    accuracy                           1.00      5200
   macro avg       1.00      1.00      1.00      5200
weighted avg       1.00      1.00      1.00      5200



In [36]:
with open(os.getcwd().replace('/notebook' , '') + "/model/surface_model.py" , "w") as f:
  f.write(m2cgen.export_to_python(svc))
print(os.path.getsize(os.getcwd().replace('/notebook' , '') + "/model/surface_model.py" ))

573
